<a href="https://colab.research.google.com/github/raulc27/python_testes/blob/master/Cap9_bs4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=c8518ca3829c5100202fe54826e1b1579ea8718c0fb6866dba4f34990ba8282d
  Stored in directory: /root/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [6]:
from bs4 import BeautifulSoup
import requests
import re

In [3]:
url = "https://www.house.gov/representatives"
text = requests.get(url).text
soup = BeautifulSoup(text,"html5lib")

In [4]:
all_urls = [a['href']
            for a in soup('a')
            if a.has_attr('href')]

In [5]:
print(len(all_urls))

967


In [7]:
# criando um conjunto (set) para eliminar repetições, após usar regex para eliminar duplicatas
regex = r"^https?://.*\.house\.gov/?$"
good_urls = [url for url in all_urls if re.match(regex, url)]
good_urls = list(set(good_urls))
print(len(good_urls))

440


In [18]:
from typing import Dict, Set
press_releases: Dict[str, Set[str]] = {}
for house_url in good_urls:
  html = requests.get(house_url).text
soup = BeautifulSoup(html, 'html5lib')
pr_links =[a['href'] for a in soup('a') if 'press releases'
in a.text.lower()]

print(f"{house_url}: {pr_links}")
press_releases[house_url] = pr_links

https://norton.house.gov/: ['/media/press-releases']


In [13]:
def paragraph_mentions(text: str, keyword: str) -> bool:
  """
  Retorna True se um <p> no texto mencionar {keyword}
  """
  soup = BeautifulSoup(text, 'html5lib')
  paragraphs = [p.get_text() for p in soup('p')]

  return any(keyword.lower() in paragraph.lower()
  for paragraph in paragraphs) 

In [14]:
# testes
text = """<body><h1>Facebook</h1><p>Twitter</p></body>"""
assert paragraph_mentions(text,"twitter")
assert not paragraph_mentions(text,"facebook")

In [15]:
# encontrando os parlamentares
for house_urls, pr_links in press_releases.items():
  for pr_link in pr_links:
    url = f"{house_url}/{pr_link}"
    text = requests.get(url).text

    if paragraph_mentions(text,'data'):
      print(f"{house_url}")
      break # fim da atividade em house_url